# Generador de retroalimentación para su posterior revisión por el docente
Este generador está pensado para pruebas de programación en las que el estudiante sube un solo fichero con su solución. Ejemplos en los que se pide que se implemente una librería, o un parte de un sistema. 

El generador va a iterar sobre todas las soluciones y va a generar un fichero HTML con el mismo nombre que el fichero de la prueba. Este es el primer paso de la coevaluación, en el segundo paso, el docente revisa y modifica la retroalimentación y calificación para poder notificarlo al estudiante.

Se espera que tengas dos carpetas preparadas para ser usadas:

1.- Carpeta **EspacioTrabajo** que contendrá el proyecto completo. En él posteriormente se copiaran la prueba del estudiante una a una y se compilará. Esta carpeta puede ser el esqueleto que se suele dar al estudiante para que trabaje durante el examen.

2.- Carpeta **pruebas** que contendrá todas las pruebas que necesitan retroalimentación. Se asume que es un sólo fichero y que vendrá con el nombre o id de cada estudiante en el nombre. Moodle tiene la opción de descargar todas las pruebas de una tarea.

In [1]:
# Importamos las liberarías necesarias
import os #Para copiar el código del estudiante al proyecto para compilar
import subprocess #Para compilar el código
from dotenv import load_dotenv #Para cargar las variables de entorno
import anthropic #Para cargar las variables necesarias para la API de Anthropic
from openai import OpenAI   #Para cargar las variables necesarias para la API de OpenAI
import shutil #Para copiar el código del estudiante al proyecto para compilar
load_dotenv()

True

In [2]:
# Puedes usar si tienes el LLM de Anthropic
# Añadimos la clave de la API, has debido crearla en OpenIA y almacenarla en el .env del entorno Anaconda.
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

MODEL_CLAUDE="claude-3-5-haiku-20241022"
claude = anthropic.Client(api_key=anthropic_api_key)

Anthropic API Key exists and begins sk-ant-


In [3]:

# Añadimos la clave de la API, has debido crearla en OpenIA y almacenarla en el .env del entorno Anaconda.
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

MODEL_GPT = 'o3-mini'
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


## Esta parte desglosa el promtp que se va a generar
* **Rúbrica general**: Contendrá las competencias que el docente quiere trabajar durante toda la asignatura (mantenibilidad, eficiencia, funcionalidad, etc).
* **Rúbrica específica**: Desglosará los métodos/funciones, su descripción y sus pesos sobre la calificación de la prueba.
* **Ejemplo de salida**: Tabla HTML que muestra un ejemplo de salida. Es interesante añadir los comentarios de retroalimentación que se esperan, y los que por experiencia, suele ser dónde fallan.

In [4]:
# Rúbrica general y
def rubricaGeneral():
    return """
Eres un evaluador de código de un examen de programación. Tu tarea es analizar la implementación del estudiante y generar una evaluación detallada que incluya una calificación numérica y una retroalimentación formativa para cada método. Para la retroalimentación ten en cuenta:
1. **Mantenimiento:**
 - Evaluar que el código sea fácilmente mantenible y extensible en el futuro.
 - Evitar el uso excesivo de bucles anidados o condiciones complejas.
 - Evitar funciones demasiado largas o complejas.
 - Evitar la duplicación de código.
 - Evitar la creación de variables locales innecesarias.
2. **Claridad:**
 - Asegurar que la estructura del código permita una lectura clara. 
 - Utilizar nombres descriptivos y claros en las variables.
 - Añadir comentarios para documentar partes complejas.
3. **Eficiencia:**
- Evitar llamadas recursivas innecesarias.
- Evitar recorridos innecesarios en estructuras de datos. 
4. **Funcionalidad:**
 - El código debe funcionar en llamadas correctas sin errores.
 - Cuando los parámetros de entrada no estén restringidos, el código de gestionar correctamente la introducción de parámetros incorrectos (por ejemplo valores NULL, número negativos, etc)
**Para la calificación de cada método**. Si la funcionalidad es correcta (punto 4 anterior), tendrá un 100% de la nota del método, si no, tendrá un máximo de un 50%. Puedes restar según los siguientes criterios hasta:
 - Un 10% si el código no es mantenible (punto 1 anterior)
 - Un 10% si el código no es claro (punto 2 anterior)
 - Un 5% si el código no es eficiente (punto 3 anterior)
**Formato de Salida:** La salida de la evaluación debe ser en HTML y contendrá:
 - Una línea con la calificación final sobre 10 y añade <br> al final.
 - Una tabla HTML con tres columnas: "Método", "Puntuación" y "Feedback".
 - Cada fila de la tabla debe corresponder a uno de los ítems evaluados."""


In [5]:
def rubricaEspecifica():
    return """Estos son los métodos a evaluar y dar una retroalimentación formativa:
 - `void poly_crear(struct Polinomio **polinomio)` (1 punto): Esta función se encarga de crear un polinomio, reservando la memoria necesaria para la estructura que lo representa y debe asignar a *p la memoria incializada. Inicializa los valores de la estructura para que esté listo para su uso posterior. En caso de no poder pedir memoria, debe finalizar el programa con código -1.
 - `int poly_grado(const struct Polinomio *polinomio)` (1 punto): Obtiene el grado de un polinomio. Devuelve -1 en caso de no poder calcularlo.
 - `int poly_coeficiente(const struct Polinomio *polinomio, int exponente)` (1 punto): Obtiene el coeficiente de un polinomio. Devuelve el coeficiente del monomio, o 0 si no existe, puede ser negativo (-3x^2).
 - `int poly_agregar(struct Polinomio *polinomio, int coeficiente, int exponente)`(2.5 puntos): Inserta un monomio en un polinomio. En caso de existir suma (ojo que coeficiente puede ser negativo o positivo, si queda a cero se elimina). En caso de no poder pedir memoria, debe finalizar el programa con código -1. En caso de no poder agregar, devuelve -1 en caso de no poder agregar. Devuelve 0 si puede agregar
 - `void poly_imprimir(const struct Polinomio *polinomio)` (1.25 puntos): Imprime los coeficientes y exponentes de un polinomio. Esta función toma un puntero a una estructura Polinomio y muestra en la salida estándar los coeficientes y exponentes del polinomio en un formato legible. Un ejemplo de salida:3x^2 2x^4 4x^5 .Otro ejemplo cuando el polinomio no existe: Polinomio no existe.
 - `void poly_destruir(struct Polinomio *polinomio)`(1.25 puntos): Destruye un polinomio liberando la memoria que ocupaba y dejando la estructura inicializada para alojar otro polinomio. Esta función se encarga de liberar la memoria que ocupaba un polinomio previamente inicializado. Se encarga de liberar la memoria de cada monomio que lo compone. Deja la estructura struct Polinomio lista ser usada de nuevo.
 - `struct Polinomio *poly_sumar(const struct Polinomio *polinomio1, const struct Polinomio *polinomio2)` (2 puntos): Esta función toma dos estructuras de polinomios como entrada y devuelve una NUEVA estructura de polinomio que representa la suma de los dos polinomios de entrada. En caso de no poder realizar la suma, se debe devolver NULL. En caso de no poder pedir memoria, debe finalizar el programa con código -1.

 Esta es la estructura de datos:
 
struct Monomio
{
	int coeficiente;
	int eexponentexp;
	struct Monomio *siguiente;
};

struct Polinomio
{
	struct Monomio *primero;
	struct Monomio *ultimo;
};"""

In [6]:
def ejemploSalida():
    return """
 ** Ejemplo de salida **:
Calificación final: 1/10

<table border="1">
  <tr>
    <th>Método</th>
    <th>Puntuación</th>
    <th>Retroalimentación</th>
  </tr>
  <tr>
    <td>poly_crear</td>
    <td>0.5 / 1</td>
    <td>
      <strong>Funcionalidad:</strong> La función no inicializa correctamente los punteros <code>primero</code> y <code>ultimo</code> después de reservar memoria. <br>
      <strong>Mantenimiento:</strong> Sin comentarios.<br>
      <strong>Claridad:</strong> Se crea una variable <code>aux</code> para pedir memoria y luego se asigna a la variable <code>*p</code>: usa directamente la variable <code>*p</code> y renombra a algo más descriptivo como <code>*polinomio</code>. Esto mejora la claridad del código con nombres descriptivos y elimina el uso de variables innecesarias que complica la compresión del código.<br>
      <strong>Eficiencia:</strong> Sin comentarios.<br>
      <em>Sugerencia Formativa:</em> Asegúrate de inicializar todos los campos de la estructura inmediatamente después de la asignación y utiliza <code>exit(-1)</code> para errores críticos en la reserva de memoria.
    </td>
  </tr>
  <tr>
    <td>poly_grado</td>
    <td>0.5 / 1</td>
    <td>
      <strong>Funcionalidad:</strong> El cálculo del grado del polinomio falla en casos en que el polinomio es nulo o la lista está vacía.<br>
      <strong>Mantenimiento:</strong>Se puede crear una función auxiliar es_polinomio_con_contenido(const struct Polinomio *polinomio) que se encargue de controlar si tiene o no contenido el polinomio. Esta función luego se puede usar en otros métodos y reduce la duplicidad de código que dificulta el mantenimiento.<br>
      <strong>Claridad:</strong> Usa nombres descriptivos para las variables, por ejemplo, en vez de usar <code>r</code>, usa <code>resultado</code>.
      <strong>Eficiencia:</strong> Es ineficiente (O(n)) recorrer la lista para encontrar el grado, ya que se puede hacer en O(1) si accedes al último del polinomio.<br>
      <em>Sugerencia Formativa:</em> La descomposición en subfunciones permite reducir la duplicidad de código y mejorar la legibilidad y el mantenimiento. 
    </td>
  </tr>
  <tr>
    <td>poly_coeficiente</td>
    <td>0.5 / 1</td>
    <td>
      <strong>Funcionalidad:</strong> La función no contempla correctamente el caso en que se solicite un exponente negativo o inexistente.<br>
      <strong>Mantenimiento:</strong>Se puede crear una función auxiliar void buscar_anterior_actual_monomio_exponente(const struct Polinomio *polinomio, int exponente, struct Monomio ** actual, struct Monomio ** anterior) que se encargue de buscar un exponente en el polinomio y de devuelva un puntero al monomio anterior y al actual. Esta función luego se puede usar en otros métodos y reduce la duplicidad de código que dificulta el mantenimiento.<br>
      <strong>Claridad:</strong> Usa nombres descriptivos para las variables, por ejemplo, en vez de usar <code>act</code>, usa <code>actual</code>.
      <strong>Eficiencia:</strong> Sin comentarios <br>
      <em>Sugerencia Formativa:</em> Revisa bien siempre los casos extremos, es dónde más suele fallar el código. ¡Usa la descomposición funcional! Esto permite reducir la duplicidad de código y mejorar la legibilidad y el mantenimiento. 
   </td>
  </tr>
  <tr>
    <td>poly_agregar</td>
    <td>1.0 / 2.5</td>
    <td>ejemp
      <strong>Funcionalidad:</strong> Al insertar un monomio, la función no suma correctamente coeficientes existentes y no elimina los nodos con coeficiente 0.<br>
      <strong>Mantenimiento:</strong> Una función así de larga y sin funciones auxiliares es compleja de mantener. Puedes sacar la inserción en cabeza y en medio/final en funciones a parte.
      <strong>Claridad:</strong> La función es muy larga y es complicada de seguir. Añadir comentarios y hacer uso de funciones auxiliares es vital para que se pueda entender.
      <strong>Eficiencia:</strong> Sin comentarios <br>
      <em>Sugerencia Formativa:</em> Una función muy larga es síntoma de que algo no va bien. Se debe siempre descomponer en bloques más simples, así haces código más mantenible.
    </td>
  </tr>
  <tr>
    <td>poly_imprimir</td>
    <td>1.25 / 1.25</td>
    <td>
      <strong>Funcionalidad:</strong> Sin comentarios <br>
      <strong>Mantenimiento:</strong> Sin comentarios <br>
      <strong>Claridad:</strong> Sin comentarios <br>
      <strong>Eficiencia:</strong> Sin comentarios <br>
      <em>Sugerencia Formativa:</em> Sin comentarios <br>
    </td>
  </tr>
  <tr>
    <td>poly_destruir</td>
    <td>0.75 / 1.25</td>
    <td>
      <strong>Funcionalidad:</strong> La función no libera correctamente todos los nodos, dejando posibles fugas de memoria. Además, debería poner a NULL el último y el primero tras eliminar el resto de nodos.<br>
      <strong>Mantenimiento:</strong> Sin comentarios <br>
      <strong>Claridad:</strong> ¡Usa nombres descriptivos! En vez de <code>aux</code>, puedes usar <code>cabeza</code> que describe mejor qué representa.<br>
      <strong>Eficiencia:</strong> Sin comentarios <br>
      <em>Sugerencia Formativa:</em> Con el depurador puedes ver si se están liberando todos los nodos. Además, es importante poner a NULL los punteros que ya no se usan para evitar errores futuros.
     </td>
  </tr>
  <tr>
    <td>poly_sumar</td>
    <td>1.0 / 2</td>
    <td>
      <strong>Funcionalidad:</strong> La función de suma no maneja correctamente los coeficientes cuando ambos polinomios tienen monomios con el mismo exponente; además, no se verifican los casos en que uno de los polinomios es NULL.<br>
      <strong>Mantenimiento:</strong> Sin comentarios <br>
      <strong>Claridad:</strong> ¡Usa nombres descriptivos! En vez de <code>aux</code>, puedes usar <code>cabeza</code> que describe mejor qué representa.<br>
      <strong>Eficiencia:</strong> Añades primero todos los de un polinomio y luego los del otro. Puedes ir añadiendo a la vez, así no recorres las dos listas al completo (cuando coinciden los exponentes, se suman y avanzan las dos a la vez).<br>
      <em>Sugerencia Formativa:</em> Revisa la salida por pantalla bien, falla en los casos básicos que se proporciona.
    </td>
  </tr>
</table>"""

     
            

## El siguiente método se encarga de compilar y devolver el prompt ya generado
* **Rúbrica general**: Contendrá las competencias que el docente quiere trabajar durante toda la asignatura (mantenibilidad, eficiencia, funcionalidad, etc).
* **Rúbrica específica**: Desglosará los métodos/funciones, su descripción y sus pesos sobre la calificación de la prueba.
* **Ejemplo de salida**: Tabla HTML que muestra un ejemplo de salida. Es interesante añadir los comentarios de retroalimentación que se esperan, y los que por experiencia, suele ser dónde fallan.

In [7]:
def prompt(userCode,compilation_output):
    return f""" {rubricaGeneral()}
                {rubricaEspecifica()}
                {ejemploSalida()}
                Evalua el siguiente código del estudiante.{compilation_output}
                {userSol}"""

In [8]:
def compilarCodigo(input_files, input_folder, MODEL):
    compilation_output=""
    userCode = ""
    try:
        for input_file in input_files:
            
            os.makedirs(os.path.dirname(input_file), exist_ok=True)
            compilation_output = ""
            relative_path = os.path.relpath(input_file, input_folder)
            dest_path = os.path.join("./EspacioTrabajo/", relative_path)
            shutil.copy(input_file, dest_path)           

        # Command to execute
        command = 'gcc ./EspacioTrabajo/principal.c ./EspacioTrabajo/PolinomioL/Polinomio.c -Wall -Wextra -Wpedantic'

    
        result = subprocess.run(
            command, shell=True, text=True,capture_output=True
        )
        
        compilation_output = "compila con ERRORES" if "error" in result.stderr or "error" in result.stdout else "compila correctamente"
        
        
    # Read the contents of all the files in the inputs_files and append to userCode
        for input_file in input_files:
            # Read the contents of the file
            with open(input_file, 'r') as f:
                content = f.read()
                # Append the content to userCode
                userCode += content + "\n ------------------\n"
    except Exception as e:
        print("An error occurred while running the compilation command:", str(e))

    return userCode,compilation_output


In [9]:
# Definimos la función input_files que recibe el nombre de la carpeta y devuelve una lista con los nombres de los archivos .c
def input_files(input_folder):
    archivos = list()
    for root, _, files in os.walk(input_folder):
        for file in files:
            if file.endswith(".c"):
                full_path = os.path.join(root, file)
                archivos.append(full_path)
    return archivos

In [10]:
def readPrueba(input_folder,MODEL):
    codigoUsario,salidaCompilacion=compilarCodigo(input_files(input_folder),input_folder,MODEL)
    return salidaCompilacion,f"""{rubricaGeneral()}
{rubricaEspecifica()}
{ejemploSalida()}

Evalua el siguiente código del estudiante que {salidaCompilacion}:

{codigoUsario}
    """
    

## Importante realizar una primera prueba y ver que la salida del prompt sea correcta
Localiza un examen de un estudiante y haz la prueba a generar el prompt. Luego introduce un error que no permita compilar y vuelve a generar. Así puebas que la generación sea correcta en ambos casos, cuando compila y cuando no.

In [11]:
a=readPrueba("./pruebas/Senador Rebeca_assignsubmission_file_/",MODEL_GPT)
print(a[0])  # Prints the first element of the tuple, e.g., 'compila correctamente'
print(a[1])  # Prints and interprets the second element of the tuple, which contains '\n'

compila correctamente

Eres un evaluador de código de un examen de programación. Tu tarea es analizar la implementación del estudiante y generar una evaluación detallada que incluya una calificación numérica y una retroalimentación formativa para cada método. Para la retroalimentación ten en cuenta:
1. **Mantenimiento:**
 - Evaluar que el código sea fácilmente mantenible y extensible en el futuro.
 - Evitar el uso excesivo de bucles anidados o condiciones complejas.
 - Evitar funciones demasiado largas o complejas.
 - Evitar la duplicación de código.
 - Evitar la creación de variables locales innecesarias.
2. **Claridad:**
 - Asegurar que la estructura del código permita una lectura clara. 
 - Utilizar nombres descriptivos y claros en las variables.
 - Añadir comentarios para documentar partes complejas.
3. **Eficiencia:**
- Evitar llamadas recursivas innecesarias.
- Evitar recorridos innecesarios en estructuras de datos. 
4. **Funcionalidad:**
 - El código debe funcionar en llamadas c

In [12]:
def EvaluarPrueba(estudiante,MODEL):
    print("----------------------")
    # Construimos el mensaje para el modelo
    salidaCompilacion, prompt=readPrueba(estudiante, MODEL)
    message = [
        {"role": "user", "content": prompt}
    ]
    # Llamamos al modelo de OpenAI o Anthropic
    result=""
    if (MODEL==MODEL_GPT):
        
        response = openai.chat.completions.create(
            messages=message,
            model=MODEL,
        )
        result= f"{salidaCompilacion}<br>\n" + response.choices[0].message.content
    elif (MODEL==MODEL_CLAUDE):
        response = claude.messages.create(
            model=MODEL_CLAUDE,
            max_tokens=5000,
            temperature=0.1,
            messages=message,
        )
        result = f"{salidaCompilacion}<br>\n" +response.content[0].text 

    return result

## Otra prueba
Escoge dos o tres estudiantes y observa si consigue calificar bien antes de continuar. Si observas que falta información o que has olvidado algo en alguna rúbrica, ahora es el momento de arreglarlo.


In [13]:
modelo = MODEL_CLAUDE
estudiante = "./pruebas/Senador Rebeca_assignsubmission_file_/"

resultado = EvaluarPrueba(estudiante, modelo)

if (resultado == None):
    print("No se ha podido compilar el código del estudiante. Por favor, revisa el código y vuelve a intentarlo.")
else:
    # Crea el nombre del archivo de salida con extensión .txt
    archivo_resultado = os.path.splitext(estudiante)[0] + "Resultado.html"
    
    # Guarda el resultado en el archivo .txt
    with open(archivo_resultado, "w") as f:
        f.write(resultado)
        print(f"Resultado guardado en: {archivo_resultado}")

----------------------
Resultado guardado en: ./pruebas/Senador Rebeca_assignsubmission_file_/Resultado.html


## Llegamos al paso final
Aquí se va iterando sobre todos los examenes y se solicita el LLM que genere la retroalimentación formativa. 

Observa que se genera un fichero resultados con el prompt de entrada y la salida.

La salida se guarda con extensión HTML y con el mismo nombre que tiene el fichero de pruebas.

In [14]:
import os
import time
# Define la carpeta donde están los archivos .c
carpeta = "./pruebas/"
archivo_centralizado = "resultados.txt"
resultados_globales = []
extension=".c"
# Función para procesar todos los archivos .c
def procesar_pruebas(carpeta, modelo):
    # Recorre todos los archivos en la carpeta
    
    for filename in os.listdir(carpeta):
        print("---------------------------")
        print(filename)
        if "." in filename:
            print("Archivo oculto, saltamos al siguiente")
            continue
            
        # Solo procesa los archivos con extensión .c
       
        
        estudiante = os.path.join(carpeta + "/", filename +"/")

        # Llama a la función EvaluarExamen y obtén el prompt
        resultado = EvaluarPrueba(estudiante, modelo)

        if (resultado == None):
            print("No se ha podido compilar el código del estudiante. Por favor, revisa el código y vuelve a intentarlo.")
        else:
            resultados_globales.append(f"{estudiante}:{resultado}")
            
            # Crea el nombre del archivo de salida con extensión .txt
            archivo_resultado = os.path.splitext(estudiante)[0] + "Resultado.html"
            
            # Guarda el resultado en el archivo .txt
            with open(archivo_resultado, "w") as f:
                f.write(resultado)
                print(f"Resultado guardado en: {archivo_resultado}")
            
    with open(archivo_centralizado, "w") as f:
        f.write("\n".join(resultados_globales))
        print(f"Resultados centralizados guardados en: {archivo_centralizado}")
    

# Llamada a la función con el modelo
procesar_pruebas(carpeta, MODEL_GPT)


---------------------------
.DS_Store
Archivo oculto, saltamos al siguiente
---------------------------
Senador Rebeca_assignsubmission_file_
----------------------
Resultado guardado en: ./pruebas//Senador Rebeca_assignsubmission_file_/Resultado.html
---------------------------
Zacarías Rusty_assignsubmission_file_
----------------------
Resultado guardado en: ./pruebas//Zacarías Rusty_assignsubmission_file_/Resultado.html
---------------------------
Calificaciones.csv
Archivo oculto, saltamos al siguiente
Resultados centralizados guardados en: resultados.txt
